In [ ]:
"""
Purpose: To develop the full method for 
creating the proofread neuron and saving it off in the database

To make this more concise and look like the function before it writes off all the data
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [6]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

import datajoint as dj

In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-04 12:14:50,737 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-04 12:14:51,030 - settings - Setting enable_python_native_blobs to True


# Importing the Neuron

In [7]:
dj.config["display.limit"] = 40
minnie.Decomposition() & "n_somas > 1"

INFO - 2021-02-04 12:16:14,947 - settings - Setting display.limit to 40


segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134765143804,0,0.25,30.00,0,0,1,=BLOB=,44570,90882,0,1,1,1,2,1,1,1,28585.105732166514,28585.105732166514,28585.106,1726.047,1726.047,1726.047,1726.047,4,0.00013993301397863443,4.0,28585.106,1,0.00013993301266750595,4.0,174766472.25666714,13946453.78167041,6113.899801322207,6113.899744036882,174766472.25666714,89.0604
864691134768966140,0,0.25,30.00,5,0,1,=BLOB=,81637,164711,0,1,1,1,2,2,5,4,61206.675450776704,40401.85629631666,4713.508,108.165,108.165,1174.845,1135.231,2,3.2676174375921285e-05,0.4,6413.497,2,0.0003118423537112436,1.0,806371663.6001841,403185831.80009204,13174.570545800676,125730.41877156628,403185831.80009204,58.3114
864691134831878402,0,0.25,30.00,0,0,1,=BLOB=,141431,288006,0,1,0,1,2,3,5,3,167837.6994974118,105050.86635215634,42893.926,114.293,114.293,1035.029,1035.029,0,0.0,0.0,19892.907,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.6019
864691134874936141,0,0.25,30.00,0,0,1,=BLOB=,64451,129722,0,2,2,1,2,3,7,3,69751.43119438276,56515.80323426945,5610.567,124.571,124.571,1568.841,1568.841,4,5.7346493563018516e-05,0.5714285714285714,41954.196,3,9.534207257839e-05,1.333,123117238.4626452,18270225.275904972,1765.0854807486746,2934.5631712891172,41039079.4875484,143.5662
864691134884769018,0,0.25,30.00,0,0,1,=BLOB=,1528011,3078802,0,7,5,2,3,20,421,129,15358033.818095537,6359831.284514944,19558.966,117.674,117.674,462.331,428.373,3853,0.0002508784682750353,9.152019002375297,8180017.673,186,0.00047102587720778387,20.715,818307027581.6089,118226275.54461034,53282.01755991983,100037.31780221165,4399500148.288219,8443.518
864691134884772346,0,0.25,30.00,0,0,1,=BLOB=,385614,805879,0,1,1,1,2,10,106,51,3416069.9194424567,1568051.5996385978,22918.888,186.605,186.605,506.188,456.498,1063,0.0003111763005639809,10.028301886792454,2126475.354,56,0.0004998882296004264,18.982,207947661950.84503,109383813.78330484,60873.36233000306,97789.82933410713,37133511

In [29]:
from pykdtree.kdtree import KDTree
import trimesh_utils as tu
import proofreading_utils as pru
import numpy as np

In [44]:
segment_id = 864691134939999587
verbose = True

# Step 1: Pulling Down the  Neurons

In [ ]:
neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

Decompressing Neuron in minimal output mode...please wait


In [ ]:
# ----- Pre-work ------

nucleus_ids,nucleus_centers = du.segment_to_nuclei(segment_id)

if verbose:
    print(f"Number of Corresponding Nuclei = {len(nucleus_ids)}")
    print(f"nucleus_ids = {nucleus_ids}")
    print(f"nucleus_centers = {nucleus_centers}")
    


original_mesh = du.fetch_segment_id_mesh(segment_id)
original_mesh_kdtree = KDTree(original_mesh.triangles_center)

In [ ]:
#Part A
filtered_neuron_list = []
cell_type_info_list = []
filtering_info_list = []

#Part B
nucleus_info_list = []

#Part C
original_mesh_faces_list = []

#Part D
keys_to_write_list = []
synapse_stats_list = []
total_error_synapse_ids_list = []

for split_index,neuron_obj in zip(neuron_split_idxs,neuron_objs):
    
    if verbose:
        print(f"\n\n-----Working on Neuron Split {split_index}-----")
        
    
    # Part A: Proofreading the Neuron
    if verbose:
        print(f"\n\n    --> Part A: Proofreading the Neuron ----")
        
        
#     nviz.visualize_neuron(neuron_obj,
#                       limb_branch_dict="all")
    
    output_dict= pru.proofread_neuron(neuron_obj,
                        plot_limb_branch_filter_with_disconnect_effect=True,
                        plot_final_filtered_neuron=False,
                        verbose=False)

    filtered_neuron = output_dict["filtered_neuron"]
    split_info = output_dict["split_info"]
    cell_type_info = output_dict["cell_type_info"]
    filtering_info = output_dict["filtering_info"]
    
    filtered_neuron_list.append(filtered_neuron)
    cell_type_info_list.append(cell_type_info)
    filtering_info_list.append(filtering_info)
    
    
    # Part B: Getting Soma Centers and Matching To Nuclei
    if verbose:
        print(f"\n\n    --> Part B: Getting Soma Centers and Matching To Nuclei ----")
        
        
    winning_nucleus_id, nucleus_info = nru.pair_segment_id_to_nuclei(neuron_obj,
                             "S0",
                              nucleus_ids,
                              nucleus_centers,
                             nuclei_distance_threshold = 20000,
                              return_matching_info = True,
                             verbose=True)

    if verbose:
        print(f"nucleus_info = {nucleus_info}")
        print(f"winning_nucleus_id = {winning_nucleus_id}")
        
    nucleus_info_list.append(nucleus_info)
    
    
    
    
        
    # Part C: Getting the Faces of the Original Mesh
    if verbose:
        print(f"\n\n    --> Part C: Getting the Faces of the Original Mesh ----")
        
    original_mesh_faces = tu.original_mesh_faces_map(original_mesh,
                                                filtered_neuron.mesh,
                                                exact_match=True,
                                                original_mesh_kdtree=original_mesh_kdtree)
    
    original_mesh_faces_list.append(original_mesh_faces)
    
#     nviz.plot_objects(recovered_mesh)






    # Part D: Getting the Synapse Information
    if verbose:
        print(f"\n\n    --> Part D: Getting the Synapse Information ----")
        
        
    (keys_to_write,
     synapse_stats,
     total_error_synapse_ids) = pru.synapse_filtering(filtered_neuron,
                    split_index,
                    segment_id=None,
                    return_synapse_filter_info = True,
                    return_synapse_center_data = False,
                    return_error_synapse_ids = True,
                    mapping_threshold = 500,
                    proof_version = 0,
                      plot_synapses=True,
                    verbose = True)
    
    
    keys_to_write_list.append(keys_to_write)
    synapse_stats_list.append(synapse_stats)
    total_error_synapse_ids_list.append(total_error_synapse_ids)
    
    
    

# Calculating all of the total synapses errored out

In [ ]:
overall_syn_error_rates = pru.calculate_error_rate(total_error_synapse_ids_list,
                        synapse_stats_list,
                        verbose=True)
overall_syn_error_rates

# Now Can Just Output the Data to The tables

In [43]:
minnie.SpineRecalculation()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,spine_version the version of the spine algorithm,"updated_spines whether or not the spines were updated (1 = yes, 0 = no)",n_spines_old number of spines before recalculation,n_spines_new number of spines after recalculation,spine_data stores the newly computes spines that were used for the classification,run_time the amount of time to run (seconds)
864691134252908410,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,15.7271
864691134316523453,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,16.0862
864691134384033872,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,15.8765
864691134428609977,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,13.5525
864691134441691943,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,14.4608
864691134484023255,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,16.0516
864691134488295985,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,16.7391
864691134488799440,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,15.8911
864691134510995985,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,15.6807
864691134570884889,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,14.7505


In [15]:
filtered_neuron_list,cell_type_info_list,filtering_info_list


([<neuron.Neuron at 0x7fca0a478f28>,
 [{'inh_exc_class': 'excitatory',
   'spine_category': 'densely_spined',
   'axon_angles': {8: {0: 136.03499403890248}, 9: {0: 158.1255683543957}},
   'axon_angle_maximum': 158.1255683543957,
   'n_axons': 2,
   'n_apicals': 0,
   'neuron_spine_density': 0.0004569936797873743,
   'n_branches_processed': 0.0004569936797873743,
   'skeletal_length_processed': 2580672.9466423863,
   'n_branches_in_search_radius': 65,
   'skeletal_length_in_search_radius': 3283740.12081483},
  {'inh_exc_class': 'excitatory',
   'spine_category': 'densely_spined',
   'axon_angles': {5: {0: 162.0842288418536}},
   'axon_angle_maximum': 162.0842288418536,
   'n_axons': 1,
   'n_apicals': 1,
   'neuron_spine_density': 0.0004696528729447189,
   'n_branches_processed': 0.0004696528729447189,
   'skeletal_length_processed': 1766348.0604305633,
   'n_branches_in_search_radius': 42,
   'skeletal_length_in_search_radius': 2036379.1463493411},
  {'inh_exc_class': 'excitatory',
   

In [16]:
nucleus_info_list

[{'nuclei_id': 500491,
  'nuclei_distance': 5641.848043206657,
  'n_nuclei_in_radius': 2,
  'n_nuclei_in_bbox': 2},
 {'nuclei_id': 496616,
  'nuclei_distance': 684.7917346315237,
  'n_nuclei_in_radius': 1,
  'n_nuclei_in_bbox': 1},
 {'nuclei_id': 492152,
  'nuclei_distance': 493.3248932391805,
  'n_nuclei_in_radius': 1,
  'n_nuclei_in_bbox': 1}]

In [17]:
original_mesh_faces_list

[array([509759, 509758, 509757, ..., 653561, 653562, 653564], dtype=uint32),
 array([142779, 142778, 142777, ..., 126947, 126924, 126940], dtype=uint32),
 array([400639, 400638, 400637, ..., 437185, 437186, 437180], dtype=uint32)]

In [20]:
keys_to_write_list
synapse_stats_list
total_error_synapse_ids_list

[{'n_valid_syn_presyn': 8,
  'n_errored_syn_presyn': 78,
  'n_valid_syn_postsyn': 142,
  'n_errored_syn_postsyn': 272},
 {'n_valid_syn_presyn': 14,
  'n_errored_syn_presyn': 72,
  'n_valid_syn_postsyn': 158,
  'n_errored_syn_postsyn': 256},
 {'n_valid_syn_presyn': 4,
  'n_errored_syn_presyn': 82,
  'n_valid_syn_postsyn': 91,
  'n_errored_syn_postsyn': 323}]